## Installations

Weights & Biases (WandB) is a machine learning development platform that allows users to track and visualize various aspects of their model training process in real-time.<br>
Link: https://wandb.ai/site

In order to use this helpful platform, first, you need to sign up in the website with the provided link. <br>
During the training you need to use your key-code to connect and see your online training results. <br>

In [1]:
!pip install wandb

     ---------------------------------------- 2.1/2.1 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 215.6/215.6 KB 138.4 kB/s eta 0:00:00
     ------------------------------------- 143.1/143.1 KB 76.6 kB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=04d1693aab3cfb839e0a4319a5ef83a95c3bc941756d895ff41518ca866ef972
  Stored in directory: c:\users\arminabd\appdata\local\pip\cache\wheels\4c\8e\7e\72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9


Ultralytics provides cutting-edge solutions for a wide range of AI tasks, including detection, segmentation, classification, tracking and pose estimation.<br>
For this specific project, I employed the YOLO8 model developed by Ultralytics. <br>
This model stands as the pinnacle of YOLO models, showcasing unparalleled performance and raising the bar in real-time detection and segmentation.<br>
Link (website): https://ultralytics.com/
Link (GitHub) : https://github.com/ultralytics

In [2]:
!pip install ultralytics

     ------------------------------------ 609.5/609.5 KB 286.3 kB/s eta 0:00:00
     ------------------------------------ 293.3/293.3 KB 452.9 kB/s eta 0:00:00
     ------------------------------------ 172.4/172.4 MB 584.9 kB/s eta 0:00:00
     ---------------------------------------- 14.9/14.9 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 7.6 MB/s eta 0:00:00
     -------------------------------------- 38.1/38.1 MB 892.1 kB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 5.6 MB/s eta 0:00:00
     ------------------------------------ 536.2/536.2 KB 415.4 kB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Attempting uninstall: seaborn
    Found existing installation: 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.0 requires numpy~=1.19.2, but you have numpy 1.24.4 which is incompatible.


This version of numpy is required

In [21]:
!pip install numpy==1.23.4

     ---------------------------------------- 14.7/14.7 MB 5.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\envs\\decode_env\\Lib\\site-packages\\~umpy\\.libs\\libopenblas64__v0.3.21-gcc_10_3_0.dll'
Consider using the `--user` option or check the permissions.



## import libraries

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from ultralytics import YOLO
import wandb
import shutil
from shutil import copytree, ignore_patterns
import os
import numpy as np

## Functions

In [9]:
def convert_xml_to_txt(label_path,xml_file,class_dict,destination_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    x=root.find('filename').text
    txt_file = x.replace('.jpg','.txt')
    txt_file = os.path.join(destination_file,txt_file)
    with open(txt_file,'w') as f:
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in class_dict:
                continue
            class_id = class_dict[class_name]
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            xmax = int(bbox.find('xmax').text)
            ymin = int(bbox.find('ymin').text)
            ymax = int(bbox.find('ymax').text)
            
            image_width = int(root.find('size/width').text)
            image_height = int(root.find('size/height').text)
            
            x_center = (xmin + xmax) / 2 / image_width if image_width != 0 else (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2 / image_height if image_height != 0 else (ymin + ymax) / 2
            width = (xmax - xmin) / image_width if image_width != 0 else (xmax - xmin)
            height = (ymax - ymin) / image_height if image_height != 0 else (ymax - ymin)
             
            line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            f.write(line)

## Dataset 1 (Fruits): Pre-processing

__Hint:__ If the datasets are zipped, unzip those before running below cells <br>
    
This is a small dataset of fruits including the ground truth for the class of fruits of: <br>
Apple: 0 <br>
Banana: 1<br>
Orange: 2<br>
and, the box around each, _x1,x2,y1,and y2_.

In [ ]:
   """ Root path for the dataset 1 """
root_path = './datasets/dataset_fruits/'
os.listdir(root_path)

In [ ]:
   """ Paths to the train & test data """
    
train_data_path = os.path.join(root_path,'train_zip/train')
test_data_path = os.path.join(root_path,'test_zip/test')

In [ ]:
   """All .xml and .jpg file names"""
    
train_data_description = os.listdir(train_data_path)
test_data_description = os.listdir(test_data_path)

In [ ]:
 """train_annotaion_file_paths and test_annotation_file_paths contains all .xml file paths
   train_image_file_paths and test_image_file_paths contains all .jpg file paths"""
    
train_annotation_file_paths = [os.path.join(train_data_path,i) for i in train_data_description if '.xml' in i]
train_image_file_paths = [os.path.join(train_data_path,i) for i in train_data_description if '.jpg' in i]

test_annotation_file_paths = [os.path.join(test_data_path,i) for i in test_data_description if '.xml' in i]
test_image_file_paths = [os.path.join(test_data_path,i) for i in test_data_description if '.jpg' in i]

print(f'length of training Data {len(train_image_file_paths)}, length of test data {len(test_image_file_paths)}')

In [ ]:
"""Creating required directories to labels"""
    
for i in ['train/labels','test/labels']:
    directory_name = os.path.join('./datasets/dataset_fruits/',i)
    if os.path.exists(directory_name):
        shutil.rmtree(directory_name)
    os.makedirs(directory_name)
    
"""Copying all images to required directories"""
copytree(train_data_path,'./datasets/dataset_fruits/train/images/',ignore = ignore_patterns('*.xml'))
copytree(test_data_path,'./datasets/dataset_fruits/test/images/',ignore = ignore_patterns('*.xml'))

In [ ]:
class_dict = {'apple':0,'banana':1,'orange':2}
dest_path = './datasets/dataset_fruits/train/labels/'
for i in train_annotation_file_paths:
    convert_xml_to_txt(train_data_path,i,class_dict,dest_path)
dest_path = './datasets/dataset_fruits/test/labels/'
for i in test_annotation_file_paths:
    convert_xml_to_txt(test_data_path,i,class_dict,dest_path)

## Training the pre-trained version of YOLO8 on Dataset 1

In [ ]:
 """yaml file path, you can check the file"""
yaml_path = "./data_FRUITS.yaml"

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
'''Training model'''
results = model.train(data=yaml_path, epochs=500, batch=4)

## Results and visualizations

In [ ]:
'''Load the model'''
model = YOLO('./train1/weights/best.pt')
img_dict = {0:'apple',1:'banana',2:'orange'}

In [ ]:
'''Load the images from each type of apples, bananas, oranges, and mixed fruits'''
path_img = './datasets/dataset_fruits/test_zip/test/apple_87.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
colors = ['r','y','b']
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
path_img = './datasets/dataset_fruits/test_zip/test/banana_89.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
path_img = './datasets/dataset_fruits/test_zip/test/orange_81.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        print(x1,confidence,label)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
path_img = './datasets/dataset_fruits/test_zip/test/mixed_22.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

## Dataset 2 (VOC): Pre-processing

__Hint:__ If the datasets are zipped, unzip those before running below cells

This is a huge dataset of indoor and outdoor images including the ground truth for 20 different classes of: <br>
  0: aeroplane<br>
  1: bicycle<br>
  2: boat<br>
  3: bus<br>
  4: car<br>
  5: motorbike<br>
  6: train<br>
  7: bird<br>
  8: cat<br>
  9: cow<br>
  10: dog<br>
  11: horse<br>
  12: sheep<br>
  13: person<br>
  14: bottle<br>
  15: chair<br>
  16: dining table<br>
  17: potted plant<br>
  18: sofa<br>
  19: tv/monitor<br>
and, the box around each, _x1,x2,y1,and y2_.


In [ ]:
   """ Root path for the dataset 1 """
root_path = './datasets/dataset_VOC/'
os.listdir(root_path)

In [ ]:
   """ Paths to the train & test data """
    
train_data_path = os.path.join(root_path,'train_zip/train')
test_data_path = os.path.join(root_path,'test_zip/test')

In [ ]:
   """All .xml and .jpg file names"""
    
train_data_description = os.listdir(train_data_path)
test_data_description = os.listdir(test_data_path)

In [ ]:
 """train_annotaion_file_paths and test_annotation_file_paths contains all .xml file paths
   train_image_file_paths and test_image_file_paths contains all .jpg file paths"""
    
train_annotation_file_paths = [os.path.join(train_data_path,i) for i in train_data_description if '.xml' in i]
train_image_file_paths = [os.path.join(train_data_path,i) for i in train_data_description if '.jpg' in i]

test_annotation_file_paths = [os.path.join(test_data_path,i) for i in test_data_description if '.xml' in i]
test_image_file_paths = [os.path.join(test_data_path,i) for i in test_data_description if '.jpg' in i]

print(f'length of training Data {len(train_image_file_paths)}, length of test data {len(test_image_file_paths)}')

In [ ]:
"""Creating required directories to labels"""
    
for i in ['train/labels','test/labels']:
    directory_name = os.path.join('./datasets/dataset_VOC/',i)
    if os.path.exists(directory_name):
        shutil.rmtree(directory_name)
    os.makedirs(directory_name)
    
"""Copying all images to required directories"""
copytree(train_data_path,'./datasets/dataset_VOC/train/images/',ignore = ignore_patterns('*.xml'))
copytree(test_data_path,'./datasets/dataset_VOC/test/images/',ignore = ignore_patterns('*.xml'))

In [ ]:
class_dict = {'aeroplane':0, 'bicycle':1, 'boat':2, 'bus':3, 
              'car':4, 'motorbike':5, 'train':6, 'bird':7, 
              'cat':8, 'cow':9, 'dog':10, 'horse':11, 'sheep':12,'person':13,
             'bottle':14, 'chair':15, 'dining table':16, 'potted plant':17, 'sofa':18, 'tv/monitor':19}

dest_path = './datasets/dataset_VOC/train/labels/'
for i in train_annotation_file_paths:
    convert_xml_to_txt(train_data_path,i,class_dict,dest_path)
dest_path = './datasets/dataset_VOC/test/labels/'
for i in test_annotation_file_paths:
    convert_xml_to_txt(test_data_path,i,class_dict,dest_path)

## Training the pre-trained version of YOLO8 on Dataset 2

In [ ]:
 """yaml file path, you can check the file"""
yaml_path = "./data_VOC.yaml"

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
'''Training model'''
results = model.train(data=yaml_path, epochs=500, batch=4)

## Results and visualizations

In [ ]:
'''Load the model'''
model = YOLO('./train2/weights/best.pt')
img_dict = {  0: 'aeroplane',  1: 'bicycle',  2: 'boat',  3: 'bus',  4: 'car',  5: 'motorbike',  6: 'train',  
            7: 'bird',  8: 'cat',  9: 'cow',  10:'dog' , 11:'horse',  12:'sheep',  13:'person',  14:'bottle',  15:'chair',  16:'dining table',
            17:'potted plant',  18:'sofa',  19:'tv/monitor'}

In [ ]:
'''Load the images from each type of apples, bananas, oranges, and mixed fruits'''
path_img = './datasets/dataset_VOC/test_zip/test/000004.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
colors = ['r','y','b']
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
'''Load the images from each type of apples, bananas, oranges, and mixed fruits'''
path_img = './datasets/dataset_VOC/test_zip/test/000069.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
colors = ['r','y','b']
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
'''Load the images from each type of apples, bananas, oranges, and mixed fruits'''
path_img = './datasets/dataset_VOC/test_zip/test/000185.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
colors = ['r','y','b']
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)

In [ ]:
'''Load the images from each type of apples, bananas, oranges, and mixed fruits'''
path_img = './datasets/dataset_VOC/test_zip/test/000297.jpg'
image = np.array(Image.open(path_img))
image_results = model(path_img)
confi_lvl_cut = 0.2
colors = ['r','y','b']
for bbox in image_results[0].boxes.data.tolist():
    x1, y1, x2, y2,confidence,label = bbox
    if confidence>=confi_lvl_cut:
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(image, img_dict[int(label)], (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image)